In [3]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-8sz1f7zg7oy0ui"
api_key = "rpa_EPOJED42G59S80Y6SKMCOI330EQU4JPPMKV2UD2W7j0uku"
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key)

#### Temasek Foundation Dataset

In [21]:
# Node dataset should ideally be concise and pure application info included ...
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.load_data import prep_tf_node 

tf_meta_prompt, test_cases, tf_metric_map = prep_tf_node(prompt_mode=True)

# Specific Metric required for TF dataset (Worth refactor the code a bit)
from methods.llm import get_groq_response
from methods.evolnode import EvolNode 
node = EvolNode(tf_meta_prompt, None, None, get_response=get_groq_response, test_cases=test_cases[:3],
                custom_metric_map=tf_metric_map) # setting manual test cases


In [44]:
# Refactor: Parallelize test input inference 
self = node 
test_inputs = [c[0] for c in test_cases[:3]]
self.code = 'def generate_prompt(project_description):\n    decision = \'Maybe\'\n    comment = \'No reasonable decision can be made without further information.\'\n    prompt = (\n        f"Given the project description {{project_description= \'{project_description}\' }}, do evaluate the grant application, make a decision (Yes, No, Maybe) and a brief comment explaining why the project is likely to be accepted or rejected. Make sure the output is in JSON format in markdown like this {{\'label\': \'str()\', \'comment\': \'str()\'}} near the top of your response."\n        )\n    return prompt'
code = self.code 

# 1. Parallelize inference 

output_per_code_per_test, errors_per_code_per_test = self.call_prompt_function_parallel(test_inputs, codes=None, max_tries=3)


# for test_input in test_inputs:
        
#     if self.meta_prompt.mode == PromptMode.CODE:   
#         _, error_msg_delta = self.call_code_function(test_input, code, file_path=None)
#         if error_msg_delta == "":
#             passed_tests += 1
#         else:
#             error_msg += error_msg_delta
            
#     elif self.meta_prompt.mode == PromptMode.PROMPT:
#         _, error_msg_delta = self.call_prompt_function(test_input, code, self.get_response)
#         if error_msg_delta == "":
#             passed_tests += 1
#         else:
#             error_msg += error_msg_delta
#     else:
#         raise ValueError(f"Unknown mode: {self.meta_prompt.mode}")
        
    

In [45]:
print(code)
# test_inputs

def generate_prompt(project_description):
    decision = 'Maybe'
    comment = 'No reasonable decision can be made without further information.'
    prompt = (
        f"Given the project description {{project_description= '{project_description}' }}, do evaluate the grant application, make a decision (Yes, No, Maybe) and a brief comment explaining why the project is likely to be accepted or rejected. Make sure the output is in JSON format in markdown like this {{'label': 'str()', 'comment': 'str()'}} near the top of your response."
        )
    return prompt


In [56]:
import signal
from typing import Dict, Any, List, Union
from methods.meta_prompt import extract_json_from_text
from methods.meta_execute import timeout_handler
import types

def _call_func_prompt_batch(input_datas: Union[list, dict], code: str, get_response: callable, max_tries: int = 3):
    """ 
    Batch prompt forward propagation
    - Returns a dictionary with indices as keys to maintain input-output correspondence
    """
    if isinstance(input_datas, dict):
        input_datas = [input_datas] 
        
    input_datas = input_datas * max_tries
    input_indices = [idx for idx, _ in enumerate(input_datas)] * max_tries
    error_messages = ["" for _ in range(len(input_indices))]

    valid_prompt_indices, valid_prompts = [], []
    for idx, input_data in zip(input_indices, input_datas):
        # Set up the timeout handler
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(3)  # Set timeout to 3 seconds
        try:
            mod = types.ModuleType('dynamic_module')
            exec(code, mod.__dict__)
            func_name = "generate_prompt" 
            prompt_func = mod.__dict__[func_name]
            prompt = prompt_func(project_description=input_data['project_description'])
            if prompt and isinstance(prompt, str):
                valid_prompt_indices.append(idx)
                valid_prompts.append(prompt)
        except Exception as e:
            error_messages[idx] += str(e)
            continue
        finally:
            signal.alarm(0)
            
    print("Valid Prompts: ", len(valid_prompts))
    # print(valid_prompts[1])
    
    responses = get_response(valid_prompts)
    
    valid_response_indices, valid_output_dicts = [], []
    for (idx, response) in zip(valid_prompt_indices, responses):
        try:
            output_dict = extract_json_from_text(response)
            if output_dict:
                valid_response_indices.append(idx)
                valid_output_dicts.append(output_dict)
        except Exception:
            error_messages[idx] += "Error extracting JSON from response"
            continue
         
    output_dicts = [{} for _ in range(len(input_indices) // max_tries)]
    for idx, output_dict in zip(valid_response_indices, valid_output_dicts):
        output_dicts[idx // max_tries] = output_dict # does not matter which one we use, anything output is fine
        
    return output_dicts, "\n".join(error_messages)




# come up with test inputs for above function 
output_dicts, error_msgs = _call_func_prompt_batch(test_inputs, code, self.get_response, max_tries=3)

Valid Prompts:  9


In [57]:
output_dicts

[{'label': 'Maybe',
  'comment': 'The project has potential, but it requires further clarification and improvement in several areas.'},
 {'label': 'Yes',
  'comment': 'The project demonstrates a clear vision for carbon-negative city infrastructure and proposes innovative solutions to address environmental concerns. The project has also secured necessary support and partners, indicating a high potential for success.'},
 {'label': 'Maybe',
  'comment': "The project's clear focus on sustainability and commercial viability is a positive, but its current ideation stage and lack of fully defined project scope of work reduce its overall feasibility. The project's potential impact may be significant, but a more detailed plan and proof of concept are needed before being fully considered for funding."},
 {},
 {},
 {},
 {},
 {},
 {}]

In [7]:
node.evolve("i1", batch_size=1) # run evolution :: Need to parallize test input inference


🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.00
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.00
  🔄 Compiled solutions:        1
  ⏱️ Time breakdown:
     :: Query time: 3.23s
     :: Evolution time: 1.25s
     :: Evaluation time: 0.40s
     :: Total time: 4.88s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
cannot access local variable 'text_rank' where it is not associated with a value
cannot access local variable 'text_rank' where it is not associated with a value
cannot access local variable 'text_rank' where it is not associated with a value



In [11]:
code

'from datetime import datetime\nimport http.client\nimport json\nimport os\nfrom typing import Dict, Any\n\n\ndef _search_google(query: str) ->Dict[str, Any]:\n    """\n    Use Serper API to search Google for information\n    \n    Args:\n        query (str): The search query\n    \n    Returns:\n        Dict[str, Any]: Parsed JSON response from the API\n    """\n    conn = http.client.HTTPSConnection(\'google.serper.dev\')\n    payload = json.dumps({\'q\': query})\n    headers = {\'X-API-KEY\': os.environ[\'SERPER_API_KEY\'], \'Content-Type\':\n        \'application/json\'}\n    try:\n        conn.request(\'POST\', \'/search\', payload, headers)\n        res = conn.getresponse()\n        data = res.read()\n        return json.loads(data.decode(\'utf-8\'))\n    except Exception as e:\n        print(f\'Error occurred during API request: {str(e)}\')\n        return {}\n    finally:\n        conn.close()\n\n\ndef search_google(query: str) ->str:\n    """ \n    Input query, return search r